In [1]:
!pip install -U finance-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import os
from tqdm import tqdm
import FinanceDataReader as fdr

# 파일별로 분류

In [3]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [4]:
sample = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

# 데이터 추출

## 현금흐름표

In [5]:
현금흐름샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(현금흐름표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 영업활동현금흐름(ifrs_CashFlowsFromUsedInOperatingActivities)
  영업활동현금흐름 = df[(df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities') | (df['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities') | (df['항목명'].str.contains('영업활동'))]
  영업활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  영업활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(영업활동현금흐름, sort=False, ignore_index=True)
  

  # 투자활동현금흐름(ifrs_CashFlowsFromUsedInInvestingActivities)
  투자활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities') | (df['항목명'].str.contains('투자활동'))]
  투자활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  투자활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(투자활동현금흐름, sort=False, ignore_index=True)

  # 재무활동현금흐름(ifrs_CashFlowsFromUsedInFinancingActivities)
  재무활동현금흐름 = df[(df['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities') | (df['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities') |(df['항목명'].str.contains('재무활동'))]
  재무활동현금흐름.rename(columns={당기:"당기"}, inplace=True)
  재무활동현금흐름.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  현금흐름샘플 = 현금흐름샘플.append(재무활동현금흐름, sort=False, ignore_index=True)


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


In [6]:
현금흐름샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동으로 인한 현금흐름,"871,088,551"
1,[060310],2021,사업보고서,3S,292,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업활동으로 인한 자산부채의 변동,"574,640,381"
2,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동 현금흐름,"-11,318,072,318"
3,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
...,...,...,...,...,...,...,...,...
14863,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,[037440],2022,1분기보고서,희림,721,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"
14865,[238490],2022,1분기보고서,힘스,292,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-254,631,420"
14866,[238490],2022,1분기보고서,힘스,292,entity00556712_udf_CF_20201022103737527_CashFl...,재무활동으로 인한 현금 유입액,"2,001,000,000"


## 손익계산서

In [7]:
손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 d설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(매출액, sort=False, ignore_index=True)


  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(당기순이익, sort=False, ignore_index=True)

  # 비지배지분(ifrs_ProfitLossAttributableToNoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests') | (df['항목코드'] == 'ifrs_ProfitLossAttributableToNoncontrollingInterests') | (df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(비지배지분, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  손익계산샘플 = 손익계산샘플.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  손익계산샘플 = 손익계산샘플.append(법인세차감전순이익, sort=False, ignore_index=True)
  

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:00<00:00, 11.83it/s]


In [8]:
손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[001040],2021,사업보고서,CJ,649,ifrs-full_Revenue,매출액,"34,484,026,554,000"
1,[000120],2021,사업보고서,CJ대한통운,493,ifrs-full_Revenue,매출액,"11,343,651,787,492"
2,[000990],2021,사업보고서,DB하이텍,261,ifrs-full_Revenue,매출액,"1,214,682,040,178"
3,[000210],2021,사업보고서,DL,649,ifrs-full_Revenue,매출액,"2,385,420,753,496"
4,[375500],2021,사업보고서,DL이앤씨,412,ifrs-full_Revenue,매출액,"7,631,650,898,582"
...,...,...,...,...,...,...,...,...
1931,[008770],2022,1분기보고서,호텔신라,478,ifrs-full_ProfitLossBeforeTax,VII. 법인세비용차감전순손익,"-7,795,875,667"
1932,[004800],2022,1분기보고서,효성,649,ifrs-full_ProfitLossBeforeTax,법인세비용차감전 계속영업이익(손실),"72,969,011,271"
1933,[298050],2022,1분기보고서,효성첨단소재,205,ifrs-full_ProfitLossBeforeTax,법인세비용차감전순이익(손실),"93,763,702,404"
1934,[298020],2022,1분기보고서,효성티앤씨,205,ifrs-full_ProfitLossBeforeTax,법인세비용차감전순이익,"159,571,477,299"


## 포괄손익계산서

In [9]:
포괄손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for path in tqdm(포괄손익계산서):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 d설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # (수익)매출액 (ifrs-full_Revenue)
  매출액 = df[(df['항목코드'] == 'ifrs_Revenue') | (df['항목코드'] == 'ifrs-full_Revenue') | (df['항목명'].isin(['매출액','(수익)매출액','매출액(수익)','수익(매출액)']))]
  매출액.rename(columns={당기:"당기"}, inplace=True)
  매출액.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(매출액, sort=False, ignore_index=True)


  # 매출총이익 (ifrs-full_GrossProfit))
  매출총이익 = df[(df['항목코드'] == 'ifrs_GrossProfit') | (df['항목코드'] == 'ifrs-full_GrossProfit')| (df['항목명'].isin(['매출총이익','(수익)매출총이익','매출총이익(손실)']))]
  매출총이익.rename(columns={당기:"당기"}, inplace=True)
  매출총이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(매출총이익, sort=False, ignore_index=True)

  # 영업이익(손실) (dart_OperatingIncomeLoss)
  영업이익 = df[(df['항목코드'] == 'dart_OperatingIncomeLoss')|(df['항목명'].isin(['영업이익','영업이익(손실)']))]
  영업이익.rename(columns={당기:"당기"}, inplace=True)
  영업이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(영업이익, sort=False, ignore_index=True)
  
  # 당기순이익(손실) (ifrs_ProfitLoss)
  당기순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLoss') | (df['항목코드'] == 'ifrs_ProfitLoss') |  (df['항목명'].isin(['당기순이익','당기순이익(손실)']))]
  당기순이익.rename(columns={당기:"당기"}, inplace=True)
  당기순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(당기순이익, sort=False, ignore_index=True)

  # 비지배지분(ifrs_ProfitLossAttributableToNoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_ProfitLossAttributableToNoncontrollingInterests') | (df['항목코드'] == 'ifrs_ProfitLossAttributableToNoncontrollingInterests') | (df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(비지배지분, sort=False, ignore_index=True)

  # 감가상각비(dart_DepreciationExpense)
  감가상각비 = df[(df['항목코드'] == 'dart_DepreciationExpense')|(df['항목명'].isin(['감가상각비']))]
  감가상각비.rename(columns={당기:"당기"}, inplace=True)
  포괄손익계산샘플 = 포괄손익계산샘플.append(감가상각비, sort=False, ignore_index=True)

  # 법인세비용차감전순이익(손실)(ifrs-full_ProfitLossBeforeTax)
  법인세차감전순이익 = df[(df['항목코드'] == 'ifrs-full_ProfitLossBeforeTax') | (df['항목코드'] == 'ifrs_ProfitLossBeforeTax') | df['항목명'].isin(['법인세차감전순이익','법인세차감전순이익(손실)'])]
  법인세차감전순이익.rename(columns={당기:"당기"}, inplace=True)
  법인세차감전순이익.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  포괄손익계산샘플 = 포괄손익계산샘플.append(법인세차감전순이익, sort=False, ignore_index=True)

  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:01<00:00,  1.44it/s]


In [10]:
포괄손익계산샘플['종목코드'] = 포괄손익계산샘플['종목코드'].apply(lambda x:x[1:-1]) 
mask = (포괄손익계산샘플['항목명'].str.contains('포괄|계속|중단|해외'))|(포괄손익계산샘플['항목코드'].str.contains('Operations|ComprehensiveIncome'))
포괄손익계산샘플 = 포괄손익계산샘플[~mask]

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,265520,2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,010240,2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,189980,2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,003280,2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,037440,2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [11]:
포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,265520,2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,010240,2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,189980,2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,003280,2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,037440,2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


## 재무상태표

In [12]:
재무상태샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(재무상태표):
  df = pd.read_csv(path, delimiter = '\t', encoding='cp949')

  당기 = ''
  if df['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기말' 
  elif df['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기말' 
  elif df['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기말' 
  elif df['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  df['사업연도'] = path[57:61] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  df['항목명'] = df['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  df['항목코드'] = df['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  df = df[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]

  전체종목 = len(df['종목코드'].unique())

  # 자산총계(ifrs_Assets)
  자산총계 = df[(df['항목코드'] == 'ifrs_Assets') | (df['항목코드'] == 'ifrs-full_Assets') | (df['항목명'].isin(['자산 총계', '자산총계']))]
  자산총계.rename(columns={당기:"당기"}, inplace=True)
  자산총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(자산총계, sort=False, ignore_index=True)

  # 자본총계(ifrs_Equity)
  자본총계 = df[(df['항목코드'] == 'ifrs_Equity') | (df['항목코드'] == 'ifrs-full_Equity') | (df['항목명'].isin(['자본총계','자본 총계']))]
  자본총계.rename(columns={당기:"당기"}, inplace=True)
  자본총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(자본총계, sort=False, ignore_index=True)

  # 비지배지분(ifrs_NoncontrollingInterests)
  비지배지분 = df[(df['항목코드'] == 'ifrs-full_NoncontrollingInterests')|(df['항목코드'] == 'ifrs_NoncontrollingInterests')|(df['항목명'].str.contains('비지배'))]
  비지배지분.rename(columns={당기:"당기"}, inplace=True)
  비지배지분.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(비지배지분, sort=False, ignore_index=True)

  # 부채총계(ifrs_Liabilities)
  부채총계 = df[(df['항목코드'] == 'ifrs_Liabilities') | (df['항목코드'] == 'ifrs-full_Liabilities') | (df['항목명'].isin(['부채총계','부채 총계']))]
  부채총계.rename(columns={당기:"당기"}, inplace=True)
  부채총계.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(부채총계, sort=False, ignore_index=True)

  # 유동자산(ifrs_CurrentAssets)
  유동자산 = df[(df['항목코드'] == 'ifrs_CurrentAssets') | (df['항목코드'] == 'ifrs-full_CurrentAssets') | (df['항목명'].isin(['유동자산', '유동 자산']))]
  유동자산.rename(columns={당기:"당기"}, inplace=True)
  유동자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(유동자산, sort=False, ignore_index=True)

  # 현금 및 현금성자산(ifrs_CashAndCashEquivalents)
  현금성자산 = df[(df['항목코드'] == 'ifrs_CashAndCashEquivalents') | (df['항목코드'] == 'ifrs-full_CashAndCashEquivalents')|df['항목명'].isin(['현금', '현금성자산', '현금 및 현금성자산', '현금및현금성자산'])|df['항목명'].str.contains('현금및현금성자산|현금 및 현금성자산')]
  현금성자산.rename(columns={당기:"당기"}, inplace=True)
  현금성자산.drop_duplicates(['종목코드','사업연도','보고서종류','항목명'])
  재무상태샘플 = 재무상태샘플.append(현금성자산, sort=False, ignore_index=True)


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 2/2 [00:02<00:00,  1.47s/it]


In [13]:
재무상태샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,[060310],2021,사업보고서,3S,292,ifrs-full_Assets,자산총계,"54,614,752,349"
1,[095570],2021,사업보고서,AJ네트웍스,763,ifrs-full_Assets,자산총계,"1,355,044,449,955"
2,[006840],2021,사업보고서,AK홀딩스,649,ifrs-full_Assets,자산총계,"4,548,736,656,905"
3,[054620],2021,사업보고서,APS홀딩스,649,ifrs-full_Assets,자산총계,"337,092,577,346"
4,[265520],2021,사업보고서,AP시스템,292,ifrs-full_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,[010240],2022,1분기보고서,흥국,292,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,[189980],2022,1분기보고서,흥국에프엔비,112,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,[003280],2022,1분기보고서,흥아해운,501,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,[037440],2022,1분기보고서,희림,721,ifrs-full_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


# 데이터 전처리

## 현금흐름표

In [14]:
현금흐름샘플['종목코드'] = 현금흐름샘플['종목코드'].apply(lambda x:x[1:-1])
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInOperatingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInInvestingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if x['항목코드'] == 'ifrs-full_CashFlowsFromUsedInFinancingActivities' else x['항목명'], axis=1)

현금흐름샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
1,060310,2021,사업보고서,3S,292,dart_AdjustmentsForAssetsLiabilitiesOfOperatin...,영업활동으로 인한 자산부채의 변동,"574,640,381"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs-full_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
...,...,...,...,...,...,...,...,...
14863,003280,2022,1분기보고서,흥아해운,501,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"
14865,238490,2022,1분기보고서,힘스,292,ifrs-full_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-254,631,420"
14866,238490,2022,1분기보고서,힘스,292,entity00556712_udf_CF_20201022103737527_CashFl...,재무활동으로 인한 현금 유입액,"2,001,000,000"


In [15]:
mask = 현금흐름샘플['항목명'].str.contains('자산|부채|기타|유출|유입|중단|연결|지급|수취|손실|제거|전|관련|조정|계속')
현금흐름샘플 = 현금흐름샘플[~mask]

In [16]:
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목명'], axis=1)

현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if 'CashFlowsFromUsedInOperatingActivities' in x['항목코드'] else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if 'CashFlowsFromUsedInInvestingActivities' in x['항목코드'] else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if 'CashFlowsFromUsedInFinancingActivities' in x['항목코드'] else x['항목코드'], axis=1)

현금흐름샘플

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [17]:
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '영업활동현금흐름' if '영업활동' in x['항목명'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '투자활동현금흐름' if '투자활동' in x['항목명'] else x['항목명'], axis=1)
현금흐름샘플['항목명'] = 현금흐름샘플.apply(lambda x: '재무활동현금흐름' if '재무활동' in x['항목명'] else x['항목명'], axis=1)

현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInOperatingActivities' if x['항목명'] == '영업활동현금흐름' else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInInvestingActivities' if x['항목명'] == '투자활동현금흐름' else x['항목코드'], axis=1)
현금흐름샘플['항목코드'] = 현금흐름샘플.apply(lambda x: 'ifrs_CashFlowsFromUsedInFinancingActivities' if x['항목명'] == '재무활동현금흐름' else x['항목코드'], axis=1)

현금흐름샘플

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [18]:
현금흐름샘플 = 현금흐름샘플.drop_duplicates()

In [19]:
현금흐름샘플['항목명'].unique()

array(['영업활동현금흐름', '투자활동현금흐름', '재무활동현금흐름'], dtype=object)

In [20]:
현금흐름샘플['항목코드'].unique()

array(['ifrs_CashFlowsFromUsedInOperatingActivities',
       'ifrs_CashFlowsFromUsedInInvestingActivities',
       'ifrs_CashFlowsFromUsedInFinancingActivities'], dtype=object)

In [21]:
현금흐름일괄 = 현금흐름샘플.drop_duplicates()
현금흐름일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


## 손익계산서

In [22]:
손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1]) 
mask = (손익계산샘플['항목명'].str.contains('포괄|계속|중단'))|(손익계산샘플['항목코드'].str.contains('continu|Continu'))
손익계산샘플 = 손익계산샘플[~mask]

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)


손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_Revenue' if x['항목명'] == '매출액' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_GrossProfit' if x['항목명'] == '매출총이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'dart_OperatingIncomeLoss' if x['항목명'] == '영업이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_ProfitLoss' if x['항목명'] == '당기순이익' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_ProfitLossAttributableToNoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'dart_DepreciationExpense' if x['항목명'] == '감가상각비' else x['항목코드'], axis=1)
손익계산샘플['항목코드'] = 손익계산샘플.apply(lambda x: 'ifrs_ProfitLossBeforeTax' if x['항목명'] == '법인세차감전순이익' else x['항목코드'], axis=1)

손익계산샘플['항목명'] = 손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [23]:
손익계산샘플 = 손익계산샘플.drop_duplicates()

In [24]:
손익계산샘플['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '감가상각비',
       '법인세차감전순이익'], dtype=object)

In [25]:
손익계산샘플['항목코드'].unique()

array(['ifrs_Revenue', 'ifrs_GrossProfit', 'dart_OperatingIncomeLoss',
       'ifrs_ProfitLoss',
       'ifrs_ProfitLossAttributableToNoncontrollingInterests',
       'dart_DepreciationExpense', 'ifrs_ProfitLossBeforeTax'],
      dtype=object)

In [26]:
손익계산샘플[손익계산샘플['항목명'] == '당기순이익(비지배)']

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
644,001040,2021,사업보고서,CJ,649,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"531,987,271,000"
645,000120,2021,사업보고서,CJ대한통운,493,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"103,537,367,210"
646,000990,2021,사업보고서,DB하이텍,261,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),0
647,000210,2021,사업보고서,DL,649,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"9,624,006,922"
648,375500,2021,사업보고서,DL이앤씨,412,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"58,791,770,098"
...,...,...,...,...,...,...,...,...
1761,008770,2022,1분기보고서,호텔신라,478,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"-718,844"
1762,004800,2022,1분기보고서,효성,649,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"6,375,168,161"
1763,298050,2022,1분기보고서,효성첨단소재,205,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"14,719,015,070"
1764,298020,2022,1분기보고서,효성티앤씨,205,ifrs_ProfitLossAttributableToNoncontrollingInt...,당기순이익(비지배),"36,723,820,139"


## 포괄손익계산서

In [27]:
포괄손익계산샘플['종목코드'] = 포괄손익계산샘플['종목코드'].apply(lambda x:x[1:-1]) 
mask = (포괄손익계산샘플['항목명'].str.contains('포괄|계속|중단|해외'))|(포괄손익계산샘플['항목코드'].str.contains('Operations|ComprehensiveIncome'))
포괄손익계산샘플 = 포괄손익계산샘플[~mask]

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목코드'] == 'ifrs-full_Revenue' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목코드'] == 'ifrs-full_GrossProfit' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목코드'] == 'dart_OperatingIncomeLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs_ProfitLoss' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목코드'] == 'ifrs-full_ProfitLoss' else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익(비지배)' if 'ProfitLossAttributableToNoncontrollingInterests' in x['항목코드'] else x['항목명'], axis=1)

포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '감가상각비' if x['항목코드'] == 'dart_DepreciationExpense' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세비용차감전순이익' if x['항목코드'] == 'ifrs-full_ProfitLossBeforeTax' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출액' if x['항목명'] == '수익(매출액)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '매출총이익' if x['항목명'] == '매출총이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '영업이익' if x['항목명'] == '영업이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '당기순이익' if x['항목명'] == '당기순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세차감전순이익(손실)' else x['항목명'], axis=1)
포괄손익계산샘플['항목명'] = 포괄손익계산샘플.apply(lambda x: '법인세차감전순이익' if x['항목명'] == '법인세비용차감전순이익' else x['항목명'], axis=1)

포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,6031,2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,9557,2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,0684,2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,5462,2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,6552,2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,1024,2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,8998,2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,0328,2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,3744,2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [28]:
포괄손익계산샘플 = 포괄손익계산샘플.drop_duplicates()
포괄손익계산샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,6031,2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,9557,2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,0684,2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,5462,2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,6552,2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,1024,2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,8998,2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,0328,2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,3744,2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [29]:
len(포괄손익계산샘플['종목코드'].unique())

1737

In [30]:
포괄손익계산샘플['항목코드'].unique()

array(['ifrs-full_Revenue', 'ifrs-full_GrossProfit',
       'dart_OperatingIncomeLoss', 'ifrs-full_ProfitLoss',
       'entity00938688_udf_IS_201711118314804_ProfitLossAttributableToAbstract',
       'ifrs-full_ProfitLossAttributableToAbstract',
       'entity00116824_udf_IS_2018320145953568_ProfitLossAttributableToAbstract',
       'entity00110750_udf_IS_2020119154120713_ProfitLossAttributableToAbstract',
       'ifrs-full_ProfitLossAttributableToNoncontrollingInterests',
       'entity00125080_udf_IS_20171110125339361_ProfitLoss',
       'entity00635134_udf_IS_2017119154220800_ProfitLoss',
       'entity00127954_udf_IS_202182172721451_ProfitLoss',
       'entity00303703_udf_IS_202031211434525_ProfitLoss',
       'entity00303703_udf_IS_2020312111337218_ProfitLossAttributableToAbstract',
       'entity00163345_udf_IS_20208716367264_ProfitLoss',
       'entity00479705_udf_IS_201932515925524_ProfitLoss',
       'entity01437186_udf_IS_202229191437133_ProfitLoss',
       'entity00140177_ud

In [31]:
포괄손익계산샘플['항목명'].unique()

array(['매출액', '매출총이익', '영업이익', '당기순이익', '당기순이익(비지배)', '비지배지분', '비지배주주지분',
       '비지배지분순이익(손실)', '비지배주주순이익', '비지배지분 당기순이익', '비지배지분에 귀속되는 당기순이익(손실)',
       '비지배지분귀속순손익', '비지배지분순이익', '비지배지분 귀속', '- 비지배지분',
       '비지배지분에 귀속되는 당기순이익', '비지배지분에 귀속될 당기순이익(손실)', '비지배지분 순이익(손실)',
       '비지배지분 귀속 순손익', '비지배주주 당기순이익', '2. 비지배지분', '비지배지분당기순이익(손실)',
       '비지배기업의 소유주에게 귀속되는 당기순이익', '비지배기준 귀속 당기순이익', '비지배지분 순이익',
       '비지배주주지분당기순이익', '비지배지분 당기순손익', '비지배지분순손익', '비지배지분 귀속 분기순이익',
       '비지배주주지분순손실', '비지배지분 귀속 당기순이익', '감가상각비', '법인세차감전순이익',
       '비지배지분에 귀속되는 분기순이익', '비지배지분에 귀속될 분기순이익(손실)', '비지배부분', '비지배기업',
       '비지배지분순손실'], dtype=object)

## 재무상태표

In [32]:
재무상태샘플['종목코드'] = 재무상태샘플['종목코드'].apply(lambda x:x[1:-1])
mask = (재무상태샘플['항목명'].str.contains('손익'))|(재무상태샘플['항목명'].isin(['비지배지분 부채', '비지배주주지분부채', '비지배지분금융부채', '유동비지배지분부채', '비지배주주지분 부채', '비지배지분 비유동부채', '비지배지분 유동부채']))
재무상태샘플 = 재무상태샘플[~mask]

재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs_Assets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목코드'] == 'ifrs-full_Assets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs_Liabilities' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목코드'] == 'ifrs-full_Liabilities' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs_Equity' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목코드'] == 'ifrs-full_Equity' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs-full_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs_CurrentAssets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목코드'] == 'ifrs-full_CurrentAssets' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs_CashAndCashEquivalents' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목코드'] == 'ifrs-full_CashAndCashEquivalents' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자산총계' if x['항목명'] == '자산 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계' if x['항목명'] == '자본 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '부채총계' if x['항목명'] == '부채 총계' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '유동자산' if x['항목명'] == '유동 자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '기타현금및현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금 및 현금성자산' else x['항목명'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '현금및현금성자산' if x['항목명'] == '현금및현금성자산-증권업' else x['항목명'], axis=1)

재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Assets' if x['항목명'] == '자산총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Liabilities' if x['항목명'] == '부채총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_Equity' if x['항목명'] == '자본총계' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_CurrentAssets' if x['항목명'] == '유동자산' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_CashAndCashEquivalents' if x['항목명'] == '현금및현금성자산' else x['항목코드'], axis=1)
재무상태샘플['항목코드'] = 재무상태샘플.apply(lambda x: 'ifrs_NoncontrollingInterests' if '비지배' in x['항목명'] else x['항목코드'], axis=1)
재무상태샘플['항목명'] = 재무상태샘플.apply(lambda x: '자본총계(비지배)' if x['항목코드'] == 'ifrs_NoncontrollingInterests' else x['항목명'], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

In [33]:
재무상태샘플

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Assets,자산총계,"54,614,752,349"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Assets,자산총계,"1,355,044,449,955"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Assets,자산총계,"4,548,736,656,905"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Assets,자산총계,"337,092,577,346"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,010240,2022,1분기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,003280,2022,1분기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,037440,2022,1분기보고서,희림,721,ifrs_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


In [34]:
재무상태샘플['항목명'].unique()

array(['자산총계', '자본총계', '자본총계(비지배)', '부채총계', '유동자산', '현금및현금성자산'],
      dtype=object)

In [35]:
재무상태샘플['항목코드'].unique()

array(['ifrs_Assets', 'ifrs_Equity', 'ifrs_NoncontrollingInterests',
       'ifrs_Liabilities', 'ifrs_CurrentAssets',
       'ifrs_CashAndCashEquivalents'], dtype=object)

# 저장

In [36]:
import datetime as dt
x = dt.datetime.now()
x.year, x.month, x.day

(2022, 7, 4)

In [37]:
현금흐름샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/현금흐름일괄.csv')
재무상태샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/재무상태표일괄.csv')
포괄손익계산샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/포괄손익일괄.csv')
손익계산샘플.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/손익일괄.csv')

# 통합

In [38]:
재무상태표일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/재무상태표일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
손익일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/손익일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
포괄손익일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/포괄손익일괄.csv',index_col = 0,  dtype={'종목코드':'str'})
현금흐름일괄 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/업데이트/(21-4, 22-1)/현금흐름일괄.csv',index_col = 0,  dtype={'종목코드':'str'})

In [39]:
현금흐름일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"871,088,551"
2,095570,2021,사업보고서,AJ네트웍스,763,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-11,318,072,318"
3,006840,2021,사업보고서,AK홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-9,766,237,791"
4,054620,2021,사업보고서,APS홀딩스,649,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"-1,568,988,010"
5,265520,2021,사업보고서,AP시스템,292,ifrs_CashFlowsFromUsedInOperatingActivities,영업활동현금흐름,"63,249,106,032"
...,...,...,...,...,...,...,...,...
14861,010240,2022,1분기보고서,흥국,292,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,309,643,228"
14862,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-75,964,141"
14863,003280,2022,1분기보고서,흥아해운,501,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-3,544,038,131"
14864,037440,2022,1분기보고서,희림,721,ifrs_CashFlowsFromUsedInFinancingActivities,재무활동현금흐름,"-2,162,502,506"


In [40]:
재무상태표일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,060310,2021,사업보고서,3S,292,ifrs_Assets,자산총계,"54,614,752,349"
1,095570,2021,사업보고서,AJ네트웍스,763,ifrs_Assets,자산총계,"1,355,044,449,955"
2,006840,2021,사업보고서,AK홀딩스,649,ifrs_Assets,자산총계,"4,548,736,656,905"
3,054620,2021,사업보고서,APS홀딩스,649,ifrs_Assets,자산총계,"337,092,577,346"
4,265520,2021,사업보고서,AP시스템,292,ifrs_Assets,자산총계,"428,720,088,928"
...,...,...,...,...,...,...,...,...
21567,010240,2022,1분기보고서,흥국,292,ifrs_CashAndCashEquivalents,현금및현금성자산,"16,028,579,778"
21568,189980,2022,1분기보고서,흥국에프엔비,112,ifrs_CashAndCashEquivalents,현금및현금성자산,"17,648,654,115"
21569,003280,2022,1분기보고서,흥아해운,501,ifrs_CashAndCashEquivalents,현금및현금성자산,"48,758,422,117"
21570,037440,2022,1분기보고서,희림,721,ifrs_CashAndCashEquivalents,현금및현금성자산,"26,045,207,278"


In [41]:
손익일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,001040,2021,사업보고서,CJ,649,ifrs_Revenue,매출액,"34,484,026,554,000"
1,000120,2021,사업보고서,CJ대한통운,493,ifrs_Revenue,매출액,"11,343,651,787,492"
2,000990,2021,사업보고서,DB하이텍,261,ifrs_Revenue,매출액,"1,214,682,040,178"
3,000210,2021,사업보고서,DL,649,ifrs_Revenue,매출액,"2,385,420,753,496"
4,375500,2021,사업보고서,DL이앤씨,412,ifrs_Revenue,매출액,"7,631,650,898,582"
...,...,...,...,...,...,...,...,...
1930,111110,2022,1분기보고서,호전실업,141,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"3,074,888,641"
1931,008770,2022,1분기보고서,호텔신라,478,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"-7,795,875,667"
1933,298050,2022,1분기보고서,효성첨단소재,205,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"93,763,702,404"
1934,298020,2022,1분기보고서,효성티앤씨,205,ifrs_ProfitLossBeforeTax,법인세차감전순이익,"159,571,477,299"


In [42]:
포괄손익일괄

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
0,6031,2021,사업보고서,3S,292,ifrs-full_Revenue,매출액,"23,477,908,383"
1,9557,2021,사업보고서,AJ네트웍스,763,ifrs-full_Revenue,매출액,"1,026,749,457,003"
2,0684,2021,사업보고서,AK홀딩스,649,ifrs-full_Revenue,매출액,"3,167,699,652,090"
3,5462,2021,사업보고서,APS홀딩스,649,ifrs-full_Revenue,매출액,"39,415,521,257"
4,6552,2021,사업보고서,AP시스템,292,ifrs-full_Revenue,매출액,"528,740,970,343"
...,...,...,...,...,...,...,...,...
22625,1024,2022,1분기보고서,흥국,292,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"3,353,383,900"
22626,8998,2022,1분기보고서,흥국에프엔비,112,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,236,152,104"
22627,0328,2022,1분기보고서,흥아해운,501,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"-371,037,287"
22628,3744,2022,1분기보고서,희림,721,ifrs-full_ProfitLossBeforeTax,법인세차감전순이익,"1,745,202,772"


In [43]:
print(f"현금흐름: {len(현금흐름일괄['종목코드'].unique())}")
print(f"손익계산서: {len(손익일괄['종목코드'].unique())}")
print(f"포괄손익계산서: {len(포괄손익일괄['종목코드'].unique())}")
print(f"재무상태표: {len(재무상태표일괄['종목코드'].unique())}")

현금흐름: 1851
손익계산서: 167
포괄손익계산서: 1737
재무상태표: 1851


In [44]:
통합재무정보 = pd.concat([재무상태표일괄, 포괄손익일괄, 손익일괄, 현금흐름일괄]).drop_duplicates()